In [1]:
import os
from pathlib import Path
import pickle


pkl_file = "part15_preprocessed.pkl"
try:
    if os.environ["assignment_2_data"] is not None:
      pkl_file = Path(os.environ["assignment_2_data"])
except:
   Exception("Problem with one of the variables")
   
assert os.path.exists(pkl_file), 'You must upload this file.'
with open(pkl_file, 'rb') as f:
  pages = pickle.load(f)  


In [2]:
from Parser import process_wiki
all_words_body, all_words_title, all_words_anchor, docs = process_wiki(pages, 'all_words')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Eden\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
from inverted_index import InvertedIndex

body_idx = InvertedIndex.read_index('body_indices', 'all_words')
title_idx = InvertedIndex.read_index('title_index', 'all_words')
anchor_idx = InvertedIndex.read_index('anchor_index', 'all_words')

In [4]:
import re
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
nltk.download('stopwords')


RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)
stopwords_frozen = frozenset(stopwords.words('english'))
def tokenize(text):
    """
    This function aims in tokenize a text into a list of tokens. Moreover, it filter stopwords.
    
    Parameters:
    -----------
    text: string , represting the text to tokenize.    
    
    Returns:
    -----------
    list of tokens (e.g., list of tokens).
    """
    list_of_tokens =  [token.group() for token in RE_WORD.finditer(text.lower()) if token.group() not in stopwords_frozen]    
    return list_of_tokens

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Eden\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
import json
with open('queries_train.json') as f:
    data = json.load(f)
# (q_id : (query_text, [relvent docs_ids]))
test_data = {q_id: d for q_id, d in enumerate(data.items())}
queries = {q_id: qf[0] for q_id, qf in test_data.items()}


In [6]:
from search import Search
from cosineSimilarity import Cosine_Similarity
from binarySimilarity import Binary_Similarity

queries_tokenize = {q_id: tokenize(q) for q_id, q in queries.items()}
search = Search()
tfidf_queries_score_train = search.get_topN_score_for_queries(queries_tokenize, title_idx, N=30, score=Binary_Similarity)
tfidf_queries_score_train = search.get_topN_score_for_queries(queries_tokenize, anchor_idx, N=30, score=Binary_Similarity)
tfidf_queries_score_train = search.get_topN_score_for_queries(queries_tokenize, body_idx, N=30, score=Cosine_Similarity)

In [19]:
results = {}
for q_id, result in tfidf_queries_score_train.items():
    results_files = list(map(lambda x: x[0], result))
    results[q_id] = list(set(results_files) & set(test_data[q_id][1]))
print(results)

{0: [], 1: [], 2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: [], 9: [], 10: [], 11: [], 12: [], 13: [], 14: [], 15: [], 16: [], 17: [], 18: [], 19: [], 20: [], 21: [], 22: [], 23: [], 24: [], 25: [], 26: [], 27: [], 28: [], 29: []}


In [14]:
from functools import reduce
from pageViews import PageViews
N = 30
pageViews_path = 'https://dumps.wikimedia.org/other/pageview_complete/monthly/2021/2021-08/pageviews-202108-user.bz2'
docs_views = PageViews(pageViews_path, docs)
print(sorted(list(docs_views.items()), key=lambda x: x[1], reverse=True)[:N])


[(17329265, 342890), (17349106, 114791), (17325260, 111797), (17333702, 59995), (17349121, 51330), (17328216, 43106), (17330081, 25231), (17325132, 24536), (17346957, 18890), (17345761, 18875), (17346443, 17744), (17348973, 16974), (17346464, 16862), (17346462, 15846), (17346456, 15036), (17337102, 14155), (17346283, 13825), (17346467, 13066), (17345999, 12783), (17331271, 12443), (17337527, 12148), (17346563, 11735), (17346433, 11660), (17333662, 8886), (17344572, 8884), (17326499, 8369), (17340975, 8280), (17327112, 8116), (17328064, 7254), (17341983, 6607)]
